This notebook translates technology subsidy in the scenarios to the required format in policy impact tables

v2 (12 May 2023) --> paste it in policy_step1_technologytax and then *ACCEPT* through TEAM UI

In [1]:
import pandas as pd

In [2]:
scenario_names = ['OMOKA', 'ELECTRICDREAMS', 'KUJINAO', 'STREETFIGHTING']

In [3]:
# you only need this for PurchasePrice and second-hand import flag
def return_technology():
    technology = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Technology.xlsx')
    return technology

In [4]:
def return_policy_impacts():
    policy_impacts = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Policy_Impacts.xlsx')
    return policy_impacts

In [5]:
def return_policy_step1_techtax():
    policy_step1 = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Policy_Step1_TechnologyTax.xlsx')
    return policy_step1

In [6]:
def return_base_rates_policy_impacts(policy_impacts, technology, base_rates={'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}):

    for techid in technology.TechID.unique().tolist():

        set_inds = policy_impacts[policy_impacts.TechID == techid].index.tolist()

        policy_impacts.loc[set_inds, 'PurchaseTax'] = base_rates['ImportDuty']*technology[technology.TechID == techid]['PurchasePrice'].item() + base_rates['ExciseDuty']*(1+base_rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item() + base_rates['VAT']*(1+base_rates['ExciseDuty'])*(1+base_rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item()

    return policy_impacts

In [7]:
# subsidies can be by FuelID, by VehTypeID.
# Narrative: Omoka focussed on growth of public e-mobility so subsidies highest for e-buses. Electric dreams has subsidies highest for e-cars and E2Ws. Street fighting = no subsidies. Kujinao has some subsidies for second-hand imported electric buses

# purhcasetax dictionary, by scenario, by fuelid (12=elec), by vehtypeid (1=bike, 2=moto, 3=car, 4=bus, 5=truck, 6=train, 7=ship, 8=plane), by SHI status (SecondHandImportFlag=0 for non-SHI, = 1 for SHI) by variable:value key value pair

# omoka - the most subisidsed electric buses
purchasetax_dict = {'OMOKA': {12: {2: {0: {'ImportDuty': 0, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            3: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            4: {0: {'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}, 1:{'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}},
                            5: {0: {'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}, 1:{'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}},
                            6: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}}}},

             # electric dreams - the most subsidised EVs.
             'ELECTRICDREAMS': {12: {2: {0: {'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}},
                            3: {0: {'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}, 1:{'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}},
                            4: {0: {'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}, 1:{'ImportDuty': 0, 'ExciseDuty': 0, 'VAT': 0}},
                            5: {0: {'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}, 1:{'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}},
                            6: {0: {'ImportDuty': 0, 'ExciseDuty': -0.1, 'VAT': 0}}}},

            # kujinao - reflects current kenyan policy as of 2023
             'KUJINAO': {12: {2: {0: {'ImportDuty': 0, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            3: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.15, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            4: {0: {'ImportDuty': 0, 'ExciseDuty': 0.1, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            5: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}},
                            6: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.1, 'VAT': 0.16}}}},

            # street fighting - no change vs 'base case' (no EV subsidies)
             'STREETFIGHTING': {12: {2: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}},
                            3: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}},
                            4: {0: {'ImportDuty': 0.3, 'ExciseDuty': 0.25, 'VAT': 0.2}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}},
                            5: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}, 1:{'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}},
                            6: {0: {'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}}}}}

In [8]:
# function to change those rates based on purchasetax_dictionary
def return_changed_policy_impacts(scenario, policy_impacts, technology):

    for fuelid in list(purchasetax_dict[scenario].keys()):

        for vehicletypeid in list(purchasetax_dict[scenario][fuelid].keys()):

            for shi_status in list(purchasetax_dict[scenario][fuelid][vehicletypeid].keys()):

                # return dictionary of rates
                rates = purchasetax_dict[scenario][fuelid][vehicletypeid][shi_status]

                # return list of techIDs to which this set of rates applies
                techids = technology[(technology.FuelID == fuelid) & (technology.VehTypeID == vehicletypeid) & (technology.SecondHandImportFlag == shi_status)]['TechID'].tolist()

                for techid in techids:

                    set_inds = policy_impacts[policy_impacts.TechID == techid].index.tolist()

                    policy_impacts.loc[set_inds, 'PurchaseTax'] = rates['ImportDuty']*technology[technology.TechID == techid]['PurchasePrice'].item() + rates['ExciseDuty']*(1+rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item() + rates['VAT']*(1+rates['ExciseDuty'])*(1+rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item()

    return policy_impacts

In [9]:
# re-write taxation policy with scenarios

for scenario_name in scenario_names:

    technology = return_technology()
    # policy_impacts = return_policy_impacts()
    policy_step1 = return_policy_step1_techtax()

    # write base rates
    policy_step1 = return_base_rates_policy_impacts(policy_step1, technology)

    # change for scenario
    policy_step1 = return_changed_policy_impacts(scenario_name, policy_step1, technology)

    # save
    policy_step1.to_excel(f'./../TEAM-Kenya_data_out/scenarios/Policy_Step1_TechnologyTax{scenario_name}.xlsx', index=False)